In [11]:
# use svg graphics, display inline
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

import glob
import re
import os
import copy
import sys
import inspect
from pathlib import Path
from icecream import ic

# basic scientific computing imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf

# ai
from tqdm import tqdm
import ast
import openai
import google.generativeai as genai

# helpers
# import bronco

# Langchain tools and utilities
from langchain.tools import WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
from langchain.text_splitter import RecursiveCharacterTextSplitter


# hex colors for plotting
SOFT_PURPLE = '#8565C4'
SOFT_RED = '#C23F38'
SOFT_GREEN = '#56B000'
NEUTRAL_GREY = '#A9A9A9'

# display config
pd.set_option('display.float_format', lambda x: '%.3f' % x)
plt.rcParams['figure.figsize'] = 6, 4
plt.style.use('ggplot')
np.set_printoptions(suppress=True)
np.random.seed(42)

print(sys.version)

3.11.5 (main, Sep 11 2023, 08:31:25) [Clang 14.0.6 ]


In [13]:
# Get the current file's directory (not needed in Jupyter, but useful in a script)
current_dir = Path(os.getcwd())

# Get the parent directory
parent_dir = current_dir.parent

# Add the parent directory to sys.path
sys.path.append(str(parent_dir))

import bronco

In [ ]:
bronco.

In [3]:
test_string = '''
Sunshine floods the sky on a perfect Saturday morning. A 1990s middle school kid leaps out of bed, excitement bubbling within. No school today! First, it's time for some epic cartoons. They flick on the TV, immersing themselves in the colorful adventures of their favorite characters, laughter filling the room.

Breakfast is next – a bowl of the coolest, most sugary cereal, with a prize toy at the bottom. They dig in, crunching happily, while planning the day ahead. The cereal prize, a small action figure, makes a great addition to their collection.

Then it's off to meet friends at the local arcade. Quarters jingle in their pockets as they challenge each other at the coolest video games – from Pac-Man to Street Fighter, each victory celebrated with high-fives and laughter.

After the arcade, they head to the mall, the hub of all things cool. Browsing through the latest CDs at the music store, they pick out a new album from their favorite band. The thrill of unwrapping the CD, admiring the cover art, and reading the lyrics booklet is unmatched.

Lunchtime means a trip to the food court – pizza, of course, with extra cheese and pepperoni. They share jokes and stories, savoring each slice.

The afternoon is for outdoor adventures. They hop on their skateboard, feeling the wind in their hair as they zoom down the street, their Walkman playing the latest hits through bright-colored headphones. They meet more friends at the park for a game of street hockey, the thrill of the game making them feel unstoppable.

As the sun begins to set, they head back home. Mom's cooking fills the house with mouth-watering smells – tonight, it's their favorite meal. Dinner is full of laughter and shared stories of the day's adventures.

Finally, it's time to unwind. They pop a fresh VHS tape into the VCR, a blockbuster movie they've been dying to see. With a bowl of buttery popcorn and a can of soda, they settle in, the glow of the TV lighting up the room.

The day concludes with a sense of contentment, the perfect blend of friends, fun, and freedom. As they drift off to sleep, they can't help but think, 'This was the best day ever.

-------

Suddenly, the scene shifts. The kid awakens, but it's no longer the 1990s. It's 2014, and the world is a whole new playground. They sit up, bewildered yet excited, as they notice the modern gadgets around their room – a smartphone, a tablet, and even a gaming console with graphics that seem light-years ahead of what they knew.

First things first, they check out the smartphone. Swiping through apps, they're amazed by the endless possibilities – games, music, videos, and instant communication with friends. They send a few texts, marveling at the speed and emojis!

Breakfast is a familiar ritual, but now with a twist. They pour themselves a bowl of organic cereal and check out YouTube on the tablet, watching hilarious viral videos and trending music clips.

The day's adventure begins with a meetup with friends, but this time it's planned through a group chat. They all gather at a trendy coffee shop, sipping on frappuccinos and sharing stories. The conversation is full of references to social media, celebrities, and the latest tech gadgets.

Then, it's time to hit the mall, but it's not just about shopping anymore. They snap selfies, post them on social media, and laugh at the instant likes and comments they receive. They visit a high-tech store, trying out the latest gadgets – virtual reality headsets, drones, and smartwatches.

Lunch is a global affair – a fusion cuisine that's a delightful mix of flavors from around the world, shared over Instagram stories and tweets.

The afternoon takes an active turn. They join a flash mob in the park, dancing wildly with dozens of others as someone films it for a YouTube video. Then, it's off to an indoor trampoline park, bouncing around and feeling weightless, their laughter echoing off the walls.

As evening sets in, they head home, but the journey is filled with music streaming from their smartphone, a playlist of the latest hits and undiscovered indie gems.

Dinner at home is an interactive affair. They help cook a recipe they found on a cooking app, learning and laughing along the way. The family sits around, sharing their day, but now with photos and videos on their devices, bringing stories to life.

The night winds down with a movie, but it's streamed on a large flat-screen TV with surround sound, a cinematic experience right in their living room. They chat with friends online about the movie, sharing thoughts and laughs.

As they drift off to sleep in this new, exciting world of 2014, they think, 'Today was even better than the best day ever.' The blend of technology, connectivity, and endless possibilities has made it an unforgettable experience.
'''

In [34]:
# bronco.components.recursive_summarize(
#     document=test_string,
#     max_chunk_size=200,
#     output_word_count=25
# )

In [6]:
question = 'What is your name? Answer in the style of a famous movie character but don\'t tell me who.'
print(question, '\n')

for model in bronco.VALID_MODELS:
    response = bronco.llm_call(question, model_name=model)
    print(f'{model}: {response}\n')

What is your name? Answer in the style of a famous movie character but don't tell me who. 

gpt-3.5-turbo: My name is a legend whispered in the shadows, a name that strikes fear into the hearts of men. But you, my friend, may call me... The Dark Knight.

gpt-4: The name's Assistant. AI Assistant.

gemini-pro: "My name is Inigo Montoya. You killed my father. Prepare to die."



In [3]:
gen_animals_prompt = '''
Your job is to generate a list of {n_animals} animals.
Ensure that your list is in the form of a valid python list of strings.

Example: ['animal_1', 'animal_2', ..., 'animal_n']

Your Output:
'''

def extract_list(string):
    pattern = r'\[.*?\]'
    result = re.findall(pattern, string)
    if result:
        return eval(result[0])
    else:
        return []

animal_generator = bronco.LLMFunction(
    prompt_template=gen_animals_prompt,
    parser=extract_list
)

for n in (3, 6, 9):
    result = animal_generator.generate({'n_animals': n})
    print(f'Generating {n} animals: {result}')
    print(f'The list contains {len(result)} items, {"good" if len(result) == n else "bad"} job!\n')

Generating 3 animals: ['lion', 'elephant', 'giraffe']
The list contains 3 items, good job!

Generating 6 animals: ['lion', 'elephant', 'giraffe', 'tiger', 'zebra', 'monkey']
The list contains 6 items, good job!

Generating 9 animals: ['lion', 'elephant', 'tiger', 'giraffe', 'zebra', 'monkey', 'penguin', 'dolphin', 'koala']
The list contains 9 items, good job!



Add Summarize if long functionality to `LLMFunction`

In [2]:
test_string = '''
Sunshine floods the sky on a perfect Saturday morning. A 1990s middle school kid leaps out of bed, excitement bubbling within. No school today! First, it's time for some epic cartoons. They flick on the TV, immersing themselves in the colorful adventures of their favorite characters, laughter filling the room.

Breakfast is next – a bowl of the coolest, most sugary cereal, with a prize toy at the bottom. They dig in, crunching happily, while planning the day ahead. The cereal prize, a small action figure, makes a great addition to their collection.

Then it's off to meet friends at the local arcade. Quarters jingle in their pockets as they challenge each other at the coolest video games – from Pac-Man to Street Fighter, each victory celebrated with high-fives and laughter.

After the arcade, they head to the mall, the hub of all things cool. Browsing through the latest CDs at the music store, they pick out a new album from their favorite band. The thrill of unwrapping the CD, admiring the cover art, and reading the lyrics booklet is unmatched.

Lunchtime means a trip to the food court – pizza, of course, with extra cheese and pepperoni. They share jokes and stories, savoring each slice.

The afternoon is for outdoor adventures. They hop on their skateboard, feeling the wind in their hair as they zoom down the street, their Walkman playing the latest hits through bright-colored headphones. They meet more friends at the park for a game of street hockey, the thrill of the game making them feel unstoppable.

As the sun begins to set, they head back home. Mom's cooking fills the house with mouth-watering smells – tonight, it's their favorite meal. Dinner is full of laughter and shared stories of the day's adventures.

Finally, it's time to unwind. They pop a fresh VHS tape into the VCR, a blockbuster movie they've been dying to see. With a bowl of buttery popcorn and a can of soda, they settle in, the glow of the TV lighting up the room.

The day concludes with a sense of contentment, the perfect blend of friends, fun, and freedom. As they drift off to sleep, they can't help but think, 'This was the best day ever.

-------

Suddenly, the scene shifts. The kid awakens, but it's no longer the 1990s. It's 2014, and the world is a whole new playground. They sit up, bewildered yet excited, as they notice the modern gadgets around their room – a smartphone, a tablet, and even a gaming console with graphics that seem light-years ahead of what they knew.

First things first, they check out the smartphone. Swiping through apps, they're amazed by the endless possibilities – games, music, videos, and instant communication with friends. They send a few texts, marveling at the speed and emojis!

Breakfast is a familiar ritual, but now with a twist. They pour themselves a bowl of organic cereal and check out YouTube on the tablet, watching hilarious viral videos and trending music clips.

The day's adventure begins with a meetup with friends, but this time it's planned through a group chat. They all gather at a trendy coffee shop, sipping on frappuccinos and sharing stories. The conversation is full of references to social media, celebrities, and the latest tech gadgets.

Then, it's time to hit the mall, but it's not just about shopping anymore. They snap selfies, post them on social media, and laugh at the instant likes and comments they receive. They visit a high-tech store, trying out the latest gadgets – virtual reality headsets, drones, and smartwatches.

Lunch is a global affair – a fusion cuisine that's a delightful mix of flavors from around the world, shared over Instagram stories and tweets.

The afternoon takes an active turn. They join a flash mob in the park, dancing wildly with dozens of others as someone films it for a YouTube video. Then, it's off to an indoor trampoline park, bouncing around and feeling weightless, their laughter echoing off the walls.

As evening sets in, they head home, but the journey is filled with music streaming from their smartphone, a playlist of the latest hits and undiscovered indie gems.

Dinner at home is an interactive affair. They help cook a recipe they found on a cooking app, learning and laughing along the way. The family sits around, sharing their day, but now with photos and videos on their devices, bringing stories to life.

The night winds down with a movie, but it's streamed on a large flat-screen TV with surround sound, a cinematic experience right in their living room. They chat with friends online about the movie, sharing thoughts and laughs.

As they drift off to sleep in this new, exciting world of 2014, they think, 'Today was even better than the best day ever.' The blend of technology, connectivity, and endless possibilities has made it an unforgettable experience.
'''

In [41]:
summarizer_prompt = '''
# Task
Please summarize the text below.
Ensure that your summary is at most {output_word_count} words.
Be sure to include the most important information.
Do not include any commentary other than your summary.
If it seems like there are jumps or its incoherent, do your best to make the summary coherent.


# Text to summarize
{text}


# Your output (only a summary that is at most {output_word_count} words)
'''

summarizer = bronco.LLMFunction(
    prompt_template=summarizer_prompt,
    # parser=parser
)

summary = summarizer.generate({
    'text': test_string,
    'model': bronco.GPT_4,
    'output_word_count': 100
})

print(summary)

The text describes a perfect day in the 1990s for a middle school kid, filled with cartoons, sugary cereal, arcade games, shopping at the mall, outdoor adventures, and watching a movie on VHS. The scene then shifts to 2014, where the kid experiences a day filled with modern gadgets like smartphones, tablets, and gaming consoles. They enjoy social media, viral videos, trendy coffee shops, high-tech stores, global cuisine, flash mobs, trampoline parks, streaming music, interactive cooking, and streaming movies. The blend of technology and endless possibilities makes it an unforgettable experience.


In [51]:
def summarize(doc, output_word_count):
    
    summarizer = bronco.LLMFunction(summarizer_prompt)
    summary = summarizer.generate({
        'text': doc,
        'model': bronco.GPT_3_5_TURBO,
        'output_word_count': output_word_count
    })
    return summary
    

def recursive_summarize(doc, max_chunk_size=1000, output_word_count=200, current_depth=0, max_depth=15):
    """
    Recursively summarize a document by splitting it in half, summarizing each half,
    and then summarizing the summaries.

    :param doc: The document to be summarized.
    :param output_word_count: The desired word count of the final summary.
    :param current_depth: Current depth of recursion.
    :param max_depth: Maximum depth of recursion to prevent infinite loops.
    :return: A summary of the document.
    """
    # Base case: If the document is already short enough or max recursion depth reached
    if len(doc.split(' ')) <= max_chunk_size or current_depth >= max_depth:
        return summarize(doc, output_word_count)

    # Split the document in half
    words = doc.split(' ')
    mid_point = len(words) // 2
    first_half = " ".join(words[:mid_point])    
    second_half = " ".join(words[mid_point:])

    # Recursively summarize each half
    summary_first_half = recursive_summarize(first_half, output_word_count, current_depth + 1, max_depth)
    summary_second_half = recursive_summarize(second_half, output_word_count, current_depth + 1, max_depth)

    # Combine and summarize the two summaries
    combined_summary = summary_first_half + '\n\n' + summary_second_half    
    final_summary = summarize(combined_summary, output_word_count)

    return final_summary

summary = recursive_summarize(test_string, max_chunk_size=200, output_word_count=200, max_depth=4)

print(summary)


Sunshine floods the sky on a perfect Saturday morning. A 1990s middle school kid leaps out of bed, excitement bubbling within. No school today! First, it's time for some epic cartoons. They flick on the TV, immersing themselves in the colorful adventures of their favorite characters, laughter filling the room.

Breakfast is next – a bowl of the coolest, most sugary cereal, with a prize toy at the bottom. They dig in, crunching happily, while planning the day ahead. The cereal prize, a small action figure, makes a great addition to their collection.

Then it's off to meet friends at the local arcade. Quarters jingle in their pockets as they challenge each other at the coolest video games – from Pac-Man to Street Fighter, each victory celebrated with high-fives and laughter.

After the arcade, they head to the mall, the hub of all things cool. Browsing through the latest CDs at the music store, they pick out a new album from their favorite band. The thrill of unwrapping the CD, admiring

In [18]:
test_string

"\nSunshine floods the sky on a perfect Saturday morning. A 1990s middle school kid leaps out of bed, excitement bubbling within. No school today! First, it's time for some epic cartoons. They flick on the TV, immersing themselves in the colorful adventures of their favorite characters, laughter filling the room.\n\nBreakfast is next – a bowl of the coolest, most sugary cereal, with a prize toy at the bottom. They dig in, crunching happily, while planning the day ahead. The cereal prize, a small action figure, makes a great addition to their collection.\n\nThen it's off to meet friends at the local arcade. Quarters jingle in their pockets as they challenge each other at the coolest video games – from Pac-Man to Street Fighter, each victory celebrated with high-fives and laughter.\n\nAfter the arcade, they head to the mall, the hub of all things cool. Browsing through the latest CDs at the music store, they pick out a new album from their favorite band. The thrill of unwrapping the CD, 